# 학원 근처 음식점 분석  

- 카카오 맵을 이용하여 학원 근처 음식점 정보를 크롤링하고 조건별 분석

## 음식점 정보 크롤링

- 이름, 카테고리, 별점, 별점 평가자 수, 리뷰수, 주소

In [ ]:
from selenium import webdriver # 드라이버를 이용하기 위해
import pyautogui
from bs4 import BeautifulSoup as bs
import time 
from datetime import datetime
from selenium.webdriver.common.keys import Keys
import pandas as pd
from tqdm import tqdm_notebook

In [ ]:
# 드라이버 연결 및 카카오맵 사이트 열기
driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(3) # 3초 기다려주자
driver.get('https://map.kakao.com/')

In [ ]:
# 검색란에 '음식점' 입력후 엔터
search = driver.find_element_by_name('q')
search.send_keys("음식점")
search.send_keys(Keys.RETURN)

In [ ]:
# 현 지도에서 다시 검색 버튼 클릭
btn_c = driver.find_element_by_css_selector('#search\.keyword\.bounds')
btn_c.click()

In [ ]:
# 장소버튼 클릭
btn_pl = driver.find_element_by_css_selector('#info\.main\.options > li.option1 > a')
btn_pl.click()

In [ ]:
# list들
name = []
ctg = []
star = []
star_count = []
review = []
address = []

In [ ]:
# 페이지 넘어가며 검색하고 리스트에 넣기
btn_n = driver.find_element_by_id('info.search.page.next') # 다음으로 넘기는 버튼

while len(name)<52:
    for i in [1,2,3,4,5]:
        # 페이지 하나씩 넘기는 버튼
        btn_p = driver.find_element_by_id('info.search.page.no{}'.format(i))
        btn_p.click()
        driver.implicitly_wait(5)
        time.sleep(0.5)
        soup = bs(driver.page_source,'html.parser')

        name_s = soup.select('#info\.search\.place\.list > li > div.head_item.clickArea > strong > a.link_name')
        name += [i.text for i in name_s]
        ctg_s = soup.select('#info\.search\.place\.list > li > div.head_item.clickArea > span')
        ctg += [i.text for i in ctg_s]
        star_s = soup.select('#info\.search\.place\.list > li > div.rating.clickArea > span.score > em')
        star += [i.text for i in star_s]
        star_count_s = soup.select('#info\.search\.place\.list > li > div.rating.clickArea > span.score > a')
        star_count += [i.text for i in star_count_s]
        review_s = soup.select('#info\.search\.place\.list > li > div.rating.clickArea > a > em')
        review += [i.text for i in review_s]
        address_s = soup.select('#info\.search\.place\.list > li > div.info_item > div.addr > p:nth-child(1)')
        address += [i.text for i in address_s]
        print(i,name_s[0].text,len(name),len(name_s)) # 진행사항 보기위해

        time.sleep(0.2)
    btn_n.click()

In [ ]:
# 잘되었는지 확인
len(name),len(ctg),len(star),len(star_count),len(review),len(address) 

In [ ]:
# 위 정보를 바탕으로 dataframe 생성
df_food = pd.DataFrame({
    'name':name, 'category':ctg, 'star':star,'star_count':star_count,'review':review, 'address':address
}) 

In [ ]:
df_food.head() # 한번더 확인

In [ ]:
df_food.to_csv('./구역_4.csv') # 저장

## 카카오 API를 이용하여 위도 경도 구하기

### 위도, 경도 구하기

In [ ]:
import requests
from urllib.parse import urlparse
import pandas as pd
import json

In [ ]:
# 주소를 입력 받아 위도,경도를 반환하는 함수
def getLatLng(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+str(addr)
    headers = {"Authorization": "KakaoAK 8d430164aae810648e75898031ed42aa"} # 카카오API 넘버(개인)
    result = json.loads(str(requests.get(url,headers=headers).text)) # 정보에서 text만 가져오기
    match_first = result['documents'][0]['address'] # 주소와 일치하는 첫번째 위도경도 가져오기
    return float(match_first['y']),float(match_first['x'])

In [ ]:
getLatLng('서울 서초구 사임당로 58 제일빌딩') # 테스트

In [ ]:
er = [] # 에러난 행 모으기
for row in tqdm_notebook(df.index): # 진행시간 확인
    try:
        df.loc[row,'lat'],df.loc[row,'lng'] = getLatLng(df.loc[row,'address'])
    except:
        er.append(row)

In [ ]:
df.iloc[er,:] #에러행 확인 에러 난 주소를 확인

In [ ]:
# 몇개 안되니까 직접 변경(아래는 카카오 맵에서 직접 찾아 변경)
# 개수가 많은 경우 다른 방법을 찾는다. 전처리 전 None값을 처리한다.
df.loc[410,'address'] = '서울 서초구 서초동 1577-11'
df.loc[471,'address'] = '서울 서초구 사임당로 58 제일빌딩'
df.loc[618,'address'] = '서울 서초구 서초중앙로 18'
df.loc[479,'address'] = '서울 서초구 서초대로50길 105'

In [ ]:
df.info() # 확인~

### 위도, 경도를 이용하여 학원과의 거리 구하기

In [ ]:
from haversine import haversine

def dis(x): # 위도와 경도로 거리 구해주는 함수 만듦
    return haversine((37.486499, 127.020664),(x.lat,x.lng)) # km

df['distance'] = df.apply(dis,axis=1) # 모든 행에 적용
df.head()

## 지표 정하기

- 별점, 별점 평가자 수, 리뷰 수를 이용하여 하나의 지표 생성
- 여러 지표를 시행해보고 결과를 확인한 뒤에 정하자
- 도메인 지식이 많이 필요하다.

### review_cnt

In [ ]:
# review_cnt로 정렬 후 그래프 그리기
df.sort_values('review_cnt',ignore_index=True,inplace=True)
plt.plot(df[1500:].index,df[1500:].review_cnt,'b-')

In [ ]:
plt.plot(df.index,np.log(df.review_cnt),'b-') # 값에 log후 그래프

In [ ]:
sns.distplot(np.log(df[(df.review_cnt>10)].review_cnt)) #displot 그래프

### score

In [ ]:
sns.displot(df.score) #displot

### eval_cnt

In [ ]:
df.sort_values('eval_cnt',ignore_index=True,inplace=True)
plt.plot(df.index,df.eval_cnt) # 정렬후 그래프

### 최종 지표(ZMS)

score * log(eval_cnt) + min-max(log(review_cnt)) ==> 100점 만점 min-max